In [21]:
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch
import os 

In [7]:
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")
os.environ["HF_HOME"] = os.getenv("HF_HOME")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation"
)
model = ChatHuggingFace(llm = llm)

d:\Code\Python\genai-projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### RunnableSequence

In [ ]:
# First generate a joke
# Then explain the joke
# Output: explanation of the joke (original joke is not returned)
prompt = PromptTemplate(
    template="Write a joke about {topic}",
    input_variables=["topic"]
)
prompt2 = PromptTemplate(
    template="Explain the following text: {text}",
    input_variables=["text"]
)
parser = StrOutputParser()

chain = RunnableSequence(prompt, model, parser, prompt2, model, parser)
print(chain.invoke({"topic": "AI"}))

This text is a play on words, using the multiple meanings of the phrase "process its emotions" to create a pun.

In a literal sense, an AI program is designed to process and analyze data, so it's reasonable to assume that the phrase "process its emotions" is meant to be taken literally, implying that the AI is having trouble with the emotional aspect of its programming.

However, in a more figurative sense, "process its emotions" is also an idiomatic expression that means to understand and deal with one's feelings. In this context, the joke is that the AI program, being a machine, is unable to truly "process its emotions" in the way that humans do, as it lacks emotional experiences and consciousness.

Therefore, the punchline "Because it was struggling to process its emotions" is a clever play on words, using the multiple meanings of the phrase to create a humorous and relatable punchline.


### RunnableParallel

In [15]:
# Create two posts for LinkedIn and tweet
prompt1 = PromptTemplate(
    template="Create a LinkedIn post about the topic: {topic}",
    input_variables=["topic"]
)
prompt2 = PromptTemplate(
    template="Create a tweet about the topic: {topic}",
    input_variables=["topic"]
)
chain = RunnableParallel({
    "linkedIn": RunnableSequence(prompt1, model, parser),
    "tweet": RunnableSequence(prompt2, model, parser)
})
print(chain.invoke({"topic": "Cricket"}))

{'linkedIn': "Here's a potential LinkedIn post about cricket:\n\n**Title:** The Unstoppable Rise of Cricket: How a Global Sport is Bridging Cultures and Careers\n\n**Post:**\n\nAs I watched the thrilling match between India and Australia at the recent ICC World Cup, I couldn't help but think about the incredible impact of cricket on our global community.\n\nFrom its humble beginnings in England in the 16th century, cricket has evolved into a beloved sport played by millions of people across the world. But beyond its entertainment value, cricket has also become a powerful tool for building bridges between cultures, fostering friendship and understanding.\n\nIn India, cricket is not just a sport - it's a way of life. The passion and dedication of Indian cricketers like MS Dhoni and Virat Kohli have inspired a nation, and their success has brought people together in a way that transcends borders.\n\nBut cricket's reach extends far beyond the playing field. The sport has also become a lucr

### RunnablePassthrough

In [ ]:
# First generate a joke
# Then explain the joke
# Expected Output: Original joke and its explanation 
prompt_create = PromptTemplate(
    template="Create a joke on the topic: {topic}",
    input_variables=["topic"]
)
prompt_explain = PromptTemplate(
    template="Explain the joke: {topic}",
    input_variables=["topic"]
)
create_chain = RunnableSequence(prompt_create, model, parser)
parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "explanation": RunnableSequence(prompt_explain, model, parser)
})
final_chain = RunnableSequence(create_chain, parallel_chain)
print(final_chain.invoke({"topic": "India"}))

{'joke': 'Why did the Indian rupee go to therapy?\n\nBecause it was feeling a little "insufficient"!\n\n(Sorry, it\'s a bit of a "dough"-lightful joke!)', 'explanation': 'This joke is a play on words, using puns to create humor. Here\'s a breakdown:\n\n- The setup "Why did the Indian rupee go to therapy?" suggests that the rupee is experiencing emotional distress, which is a common reason for going to therapy.\n- The punchline "Because it was feeling a little \'insufficient\'" is where the joke gets its humor. In this context, "insufficient" has a double meaning:\n  - In economic terms, an insufficient amount of money or resources can be a problem.\n  - The word "insufficient" sounds similar to "not enough" or "lacking," which can also refer to something being insufficient in terms of supply or amount.\n- The second part of the joke, "a little \'dough\'-lightful," adds another layer of wordplay. "Dough" is a pun on the phrase "doughlightful," which sounds similar to "delightful." Howev

### RunnableLambda

In [ ]:
# Use custom function count_words in Runnablelambda

def count_words(sentence):
    return len(sentence.split(' '))

prompt_create = PromptTemplate(
    template="Create a joke on the topic: {topic}",
    input_variables=["topic"]
)
create_chain = RunnableSequence(prompt_create, model, parser)
parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "word_count": RunnableLambda(count_words)
})
final_chain = RunnableSequence(create_chain, parallel_chain)

print(final_chain.invoke({"topic": "India"}))

{'joke': 'Why did the Indian rupee go to therapy?\n\nBecause it was feeling a little "devalued"!', 'word_count': 14}


### RunnableBranch

In [23]:
# Conditional - based on no. of words 
#  If > 500 words, summarize it. Otherwise print as it is.

prompt = PromptTemplate(
    template="Create a post (between 400-600 words) on the topic: {topic}",
    input_variables=["topic"]
)
create_chain = RunnableSequence(prompt, model, parser)

prompt2 = PromptTemplate(
    template="Summarize the topic: {topic}",
    input_variables=["topic"]
)
branch_chain = RunnableBranch(
    (lambda x: len(x.split()) > 500, RunnableSequence(prompt2, model, parser)),
    RunnablePassthrough()
)
final_chain = RunnableSequence(create_chain, branch_chain)

print(final_chain.invoke({"topic": "Russia v/s Ukraine"}))


The conflict between Russia and Ukraine has deep historical, cultural, and political roots. The situation escalated in 2014 when Russia annexed Crimea and supported separatist movements in eastern Ukraine, resulting in significant human suffering, displacement, and economic losses.

The relationship between Russia and Ukraine dates back to the Kievan Rus', a medieval East Slavic state. Ukraine was a key component of the Rus' state and later became a major center of Eastern Orthodox Christianity. In the 18th century, Ukraine was annexed by the Russian Empire and remained under Russian rule until the Russian Revolution.

In 2010, Ukraine's pro-Russian president was ousted, leading to a shift in Ukraine's foreign policy towards the European Union and the United States. Russia responded by tightening its grip on Crimea and supporting separatist movements in eastern Ukraine, resulting in a conflict that has resulted in the deaths of over 13,000 people.

The international community has been 